In [ ]:
PYMEPIX_IP = '127.0.0.1'
PYMEPIX_PORT = 5056
SERVE_PORT = 5010

In [ ]:
# Imports
import time
import numpy as np
import pandas as pd

import panel as pn
import param
#pn.extension('altair', 'tabulator')

import asyncio

from IPython.lib import backgroundjobs as bg

from pymepix.channel.client import Client
from pymepix.channel.channel_types import ChannelDataType, Commands
import pymepix.config.load_config as cfg

import asyncio
from tornado import gen
from tornado.locks import Condition

import holoviews as hv
from holoviews import opts
from holoviews import streams
#import hvplot.pandas
from holoviews.streams import Pipe
from bokeh.plotting import show
hv.extension('bokeh')
#pd.options.plotting.backend = 'holoviews'

## Pymepix Client connection for data receiving ##

In [ ]:
channel_address = tuple(cfg.default_cfg.get('tcp_channel', ['127.0.0.1', 5056]))

client = Client(channel_address, None,)
data_queue = client.get_queue()

condition = Condition()

jobs = bg.BackgroundJobManager()

## Globals ##

In [ ]:
H_VMI = np.zeros((255, 255), dtype=float)
roi_H_VMI = H_VMI

bins_tof = np.linspace(0, 10, 500)
x_axe = bins_tof[:-1]
tof_accu = np.zeros((len(bins_tof)-1), dtype=float)
record_tof_accu = np.zeros((len(bins_tof)-1), dtype=float)
chunks_counter=0

roi_scan_data = []

bins_VMI = (range(0, 256), range(0, 256))

Element_ROI = {
              "roi1": [(5.57, 5.67), np.zeros((255, 255)), 'red'],
              "roi2": [(5.67, 5.77), np.zeros((255, 255)), 'green'],
              "roi3": [(6.4, 6.5), np.zeros((255, 255)), 'blue'],
              "roi4": [(8.15, 8.25), np.zeros((255, 255)), 'orange'],
              }


pipe_roi_scan = Pipe(np.zeros((1,len(Element_ROI)+1)))
pipe_tof = Pipe(data=np.zeros((len(bins_tof)-1), dtype=float))
pipe_histo = Pipe(data=np.zeros((256,256)))

chunk_data=None

DATA_FILTER = [ChannelDataType.TOF.value, ChannelDataType.COMMAND.value]

pipe_roi_scan.send(np.zeros((1,len(Element_ROI)+1)))
pipe_tof.send(H_VMI)
pipe_histo.send(tof_accu)

## Panel entities with callbacks ##

In [ ]:
def on_roi_select_change(Event):
    
    numpy_array = np.array(roi_scan_data)
    if len(numpy_array) > 0:
        pipe_roi_scan.send(numpy_array)
    else:
        pipe_roi_scan.send(np.zeros((1,len(Element_ROI)+1)))
    
    if roi_select.value == []:
        pipe_tof.send(H_VMI)
    else:
        roi_H_VMI = np.zeros((255, 255), dtype=float)
        for roi_key, roi_val in Element_ROI.items():
            if roi_key in roi_select.value:
                roi_H_VMI += roi_val[1]
        pipe_tof.send(roi_H_VMI)
        
    pipe_histo.send(tof_accu/max(tof_accu.max(),1))
    

def reset_button_callback(Event):
    global H_VMI, tof_accu, roi_scan_data, record_tof_accu
    H_VMI = np.zeros((255, 255), dtype=float)
    tof_accu = np.zeros((len(bins_tof)-1), dtype=float)
    roi_scan_data = []
    record_tof_accu = np.zeros((len(bins_tof)-1), dtype=float)
    
    
    for roi_val in Element_ROI.values():
        roi_val[1] = np.zeros((255, 255), dtype=float)
    
    pipe_roi_scan.send(np.zeros((1,len(Element_ROI)+1)))
    pipe_tof.send(H_VMI)
    pipe_histo.send(tof_accu)

roi_select = pn.widgets.MultiChoice(
    name='ToF ROI', value=[list(Element_ROI.keys())[0]],
    options=list(Element_ROI.keys()),)

reset_button = pn.widgets.Button(name='Reset', button_type='primary')
reset_button.on_click(reset_button_callback)

roi_select.param.watch(on_roi_select_change, 'value')

## The routine where the data processing takes place ##

In [ ]:
def main_loop(data_filter=DATA_FILTER):
    
    global H_VMI, roi_H_VMI
    global tof_accu
    global x_axe
    global roi_scan_data
    global roi_select
    global chunk_data
    global record_tof_accu, chunks_counter
    
    while True:
        data = data_queue.get()

        if data['type'] not in data_filter:
            continue

        chunk_data = data

        if data['type'] == ChannelDataType.TOF.value:

            tof_data = chunk_data['data']
            mcs_tof = tof_data[3]*1e6
            H_VMI += np.histogram2d(tof_data[1], tof_data[2], bins=bins_VMI)[0]
            for roi_key, roi_val in Element_ROI.items():
                roi_indx = np.logical_and(mcs_tof>roi_val[0][0], mcs_tof<roi_val[0][1])
                roi_val[1] += np.histogram2d(tof_data[1][roi_indx], tof_data[2][roi_indx], bins=bins_VMI)[0]


            if roi_select.value == []:
                roi_H_VMI = H_VMI
                #roi_H_VMI = pipe_tof.send(H_VMI)                
            else:
                roi_H_VMI = np.zeros((255, 255), dtype=float)
                for roi_key, roi_val in Element_ROI.items():
                    if roi_key in roi_select.value:
                        roi_H_VMI += roi_val[1]
                #pipe_tof.send(roi_H_VMI)

            chunk_tof_data = np.histogram(tof_data[3]*1e6, bins=bins_tof)[0]
            tof_accu += chunk_tof_data
            record_tof_accu += chunk_tof_data/len(np.unique(tof_data[0]))
            #pipe_histo.send(tof_accu/tof_accu.max())
            chunks_counter += 1
            
            #condition.notify()


        if chunk_data['type'] == ChannelDataType.COMMAND.value:            
            if chunk_data['data'] == Commands.START_RECORD.value:
                H_VMI = np.zeros((255, 255), dtype=float)
                tof_accu = np.zeros((len(bins_tof)-1), dtype=float)
                for roi_val in Element_ROI.values():
                    roi_val[1] = np.zeros((255, 255), dtype=float)
                record_tof_accu = np.zeros((len(bins_tof)-1), dtype=float)
                chunks_counter = 0

            elif chunk_data['data'] == Commands.STOP_RECORD.value:
                scan_dataset = [len(roi_scan_data)]

                chunk_mean_tof_data = record_tof_accu/chunks_counter

                for roi in Element_ROI.values():
                    roi_indxs=np.logical_and(x_axe>roi[0][0], x_axe<roi[0][1])                    
                    scan_dataset.append(np.sum(chunk_mean_tof_data[roi_indxs]))

                roi_scan_data.append(scan_dataset)
                
                #condition.notify()
                #numpy_array = np.array(roi_scan_data)
                #pipe_roi_scan.send(numpy_array)


async def draw_callback():
    
    global roi_scan_data
    global roi_H_VMI
    global tof_accu
    
    while True:
    
        #await condition.wait()
        await asyncio.sleep(0.5)

        numpy_array = np.array(roi_scan_data)
        if len(numpy_array) > 0:
            pipe_roi_scan.send(numpy_array)
        else:
            pipe_roi_scan.send(np.zeros((1,len(Element_ROI)+1)))

        pipe_histo.send(tof_accu/max(tof_accu.max(),1))

        pipe_tof.send(roi_H_VMI)
    
    
            

jobs.new(main_loop)

loop = asyncio.get_event_loop()
asyncio.run_coroutine_threadsafe(draw_callback(), loop)

jobs.status()

## Panel GUI, server start ##

In [ ]:
selected_rois = None

def make_roi_scan_plot(data):
    global roi_select
    if roi_select.value != [] and data is not None:
        
        roi_indxs = [roi_select.options.index(i) for i in roi_select.value]
        
        xmax = np.max(data[:,0])
        xmin = np.min(data[:,0])
        ymax = np.max(data[:,np.array(roi_indxs)+1])
        ymin = np.min(data[:,np.array(roi_indxs)+1])

        if xmax==xmin:
            xlimits = (xmax-1,xmax+1,)
        else:
            xrange = xmax-xmin
            xlimits = (xmin - xrange*0.1,xmax+xrange*0.1,)
            
        if ymax==ymin:
            ylimits = (ymax-1,ymax+1,)
        else:
            yrange = ymax-ymin
            ylimits = (ymin - yrange*0.1,ymax+yrange*0.1,)
        
        colors=[i[2] for i in list(Element_ROI.values())]
        curves=[hv.Curve((data[:,0],data[:,i+1])).opts(xlabel='scan num.', ylabel='amplitude, [au]', width=600, height=450, show_grid=True, tools=['hover'], 
                                                       color=colors[i], xlim=xlimits, ylim=ylimits, axiswise=True, framewise=True,)*hv.Scatter((data[:,0],data[:,i+1])).opts(xlabel='scan num.', 
                                                       ylabel='amplitude, [au]', width=600, height=450, show_grid=True, tools=['hover'], color=colors[i], xlim=xlimits, ylim=ylimits, axiswise=True, 
                                                       framewise=True, size=10) for i in roi_indxs]

        return hv.Overlay(curves).opts( axiswise=True, framewise=True,).redim.range(Sample=(0,10))
    else: 
        return hv.Curve([]).opts(xlabel='scan num.', ylabel='amplitude, [au]', width=600, height=450, show_grid=True, tools=['hover'],  axiswise=True, framewise=True,)
    
def make_image_plot(data):
    H_VMI = data.T
    return hv.Image(H_VMI, bounds=[bins_VMI[0][0], bins_VMI[1][0], bins_VMI[0][-1], bins_VMI[1][-1]],).opts(xlabel='h', ylabel='v', axiswise=True, logz=True, clim=(1e-3, None), cmap="jet", frame_height=400, frame_width=400)

def make_histo_plot(data):
    global roi_select, selected_rois
    roi_indxs = [roi_select.options.index(i) for i in roi_select.value]
    
    selected_rois = [(Element_ROI[i][0], Element_ROI[i][2]) for i in roi_select.value]

    mass_rois = [hv.Rectangles((i[0], 0, i[1], 1)).opts(alpha=0.3, color=c) for i, c in selected_rois]
    
    return hv.Curve((x_axe, data)).opts(xlabel='ToF (µs)', ylabel='normalized amplitude', axiswise=True, width=1100, show_grid=True, tools=['hover'], ylim=(0,1)) * hv.Overlay(mass_rois)

pn.serve(pn.Row(pn.Column(roi_select, reset_button),
                pn.Column(pn.Row(hv.DynamicMap(make_roi_scan_plot, streams=[pipe_roi_scan]),
                                 hv.DynamicMap(make_image_plot, streams=[pipe_tof])),
                          hv.DynamicMap(make_histo_plot,  streams=[pipe_histo]))),)
#         port=SERVE_PORT)